## 모듈 불러오기

정상 - 하나로
결막염
백내장

In [2]:
!pip install split-folders


In [3]:
from PIL import Image
import requests
import numpy as np
import matplotlib.pyplot as plt
import splitfolders
import tensorflow as tf
import os

from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Dense

## 데이터 정의


### 압축 풀기


In [5]:
!unzip -q "/content/drive/MyDrive/CV_project/data/안구질환/개_질환_최종.zip"


### train, test, valid split

In [6]:
splitfolders.ratio('/content/개', output='output_dir', ratio=(0.8, 0.1, 0.1))  # train/val/test = 8:1:1


Copying files: 68875 files [00:14, 4599.53 files/s]


In [7]:
train_path = 'output_dir/train'
val_path = 'output_dir/val'
test_path = 'output_dir/test'

### 정규화

In [35]:
img_width, img_height = 400, 400
batch_size = 32
num_classes = 10
last_activation = 'softmax'

train_ds = image_dataset_from_directory(
    train_path,
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    label_mode='int'
)

valid_ds = image_dataset_from_directory(
    val_path,
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    label_mode='int'
)

test_ds = image_dataset_from_directory(
    test_path,
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    label_mode='int'
)

NameError: name 'img_width' is not defined

In [ ]:
# labels = train_path.class_names
# labels

In [ ]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)

# train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# for images, labels in train_ds.take(1):
#     print(images.shape, labels.shape, labels.dtype)


(32, 400, 400, 3) (32,) <dtype: 'int32'>


## 모델 구축


### 하이퍼파라미터 값 정의

In [24]:
# 하이퍼파라미터 설정
nam_classes = 10      # 데이터셋의 클래스 개수 (안구 질환은 10개)
input_shape = (400, 400, 3)  # 입력 이미지 형태 (배치 크기, 높이, 너비, 채널 수)
learning_rate = 0.001  # 학습률
batch_size = 32        # 배치 크기
epoch_num = 30        # 학습 에포크 수
num_heads = 8         # Multi-Head Attention 헤드 개수
trans_num = 8         # Transformer 블록 반복 횟수
img_patch_size = 16     # 이미지 패치 크기 (가로 및 세로)
p_i_num = int(400 / 4 * 400 / 4) + 1  # Positional Embedding 입력 크기 (패치 개수 + 1)
proj_num = 32        # 투영 벡터 차원
class_num = 10       # 분류할 클래스 개수

In [34]:
# 인코딩 값 정의
position_input = [range(64) for i in range(68875 )]
position_input = np.array(position_input)
position_input = np.reshape(position_input, (68875 ,64,1))

# 모델 정의
i = tf.keras.Input(shape=input_shape)
p_i = tf.keras.Input(shape=(p_i_num,1))
out_patch = tf.image.extract_patches(images=i,
            sizes=[1, img_patch_size, img_patch_size, 1],
            strides=[1, img_patch_size, img_patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID")

# out_patch: (배치 크기, 패치 개수, 채널 수)
# p_out: (배치 크기, 패치 개수, 채널 수)

# out = tf.keras.layers.Reshape([-1,img_patch_size*img_patch_size*3])(out_patch)
out_patch = tf.keras.layers.Reshape([-1, img_patch_size*img_patch_size*3])(out_patch)
print(out_patch.shape())

p_out = tf.keras.layers.Embedding(p_i_num, proj_num)(p_i)
# p_out = tf.keras.layers.Reshape([-1,proj_num])(p_out)
p_out = tf.keras.layers.Reshape([-1, proj_num])(p_out)
out = tf.keras.layers.Dense(proj_num)(out_patch)
out = tf.keras.layers.Add()([out, p_out])

# for _ in range(trans_num):
#     out_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(out)
#     a_out = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
#                                                key_dim=proj_num,
#                                                dropout=0.1)(out_1, out_1)
#     out_2 = tf.keras.layers.Add()([a_out, out])
#     out_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(out_2)
#     out_3 = tf.keras.layers.Dense(proj_num *2, activation=tf.nn.gelu)(out_3)
#     out_3 = tf.keras.layers.Dropout(0.1)(out_3)
#     out_3 = tf.keras.layers.Dense(proj_num, activation=tf.nn.gelu)(out_3)
#     out_3 = tf.keras.layers.Dropout(0.1)(out_3)
#     out = tf.keras.layers.Add()([out_3, out_2])

# out = tf.keras.layers.LayerNormalization(epsilon=1e-6)(out)
# out = tf.keras.layers.Flatten()(out)
# out = tf.keras.layers.Dropout(0.5)(out)
# out = tf.keras.layers.Dense(1024, activation=tf.nn.gelu)(out)
# out = tf.keras.layers.Dropout(0.1)(out)
# out = tf.keras.layers.Dense(512, activation=tf.nn.gelu)(out)
# out = tf.keras.layers.Dropout(0.1)(out)
# out = tf.keras.layers.Dense(class_num)(out)

# vit_model = tf.keras.Model(inputs=[i,p_i], outputs=[out])

# # 모델 요약
# vit_model.summary()


TypeError: 'TensorShape' object is not callable

In [ ]:
# 모델 구조
tf.keras.utils.plot_model(vit_model,show_shapes=True)

In [ ]:
vit_image_model=  tf.keras.Model(inputs=vit_model.inputs, outputs=vit_model.layers[1].output)
img=vit_image_model.predict([x_train[0:2],position_input[0:2]])
img.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_train[0]
plt.imshow(image.astype("uint8"))
plt.axis("off")

In [ ]:
plt.figure(figsize=(4, 4))
for i in range(8):
    for j in range(8):
        ax = plt.subplot(8, 8, i*8 +j+1)
        sub_img=np.reshape(img[0,i,j,:],(4,4,3))
        plt.imshow(sub_img.astype("uint8"))
        plt.axis("off")

In [ ]:
vit_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy")])

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping # 모델 저장, 조기종료

checkpoinat_cb = ModelCheckpoint('/content/drive/MyDrive/CV_project/data/안구질환/best-cnn-model.h5')   # 최적 검증 점수를 만드는 모델 자동 저장
early_stopping_cb = EarlyStopping(patience=2, restore_best_weights=True)  # 과대 적합 방지 및 , 필요하지 않은 훈련 하지 않도록 조기 종료


vit_model.fit(x=[x_train,position_input],
              y=y_train,
              batch_size=batch_size,
              epochs=epoch_num,
              callbacks=[checkpoinat_cb, early_stopping_cb]
              validation_split=0.3)